In [ ]:
import os
import re
import pandas as pd

In [ ]:
# Функция для чтения текстовых файлов и запись в массив

def read_files(directory):
  texts = []

  for filename in os.listdir(directory):
    if filename.endswith('.txt'):
      with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
        texts.append(file.read())

  return texts

In [ ]:
# Функция редактирование файлов. Убирает спикеров

def del_speakers(lst_files: list) -> list:
  list_del_speakers = []
  pattern = r'(S|s)(P|p).?[0-5]\s?:*\s?'
  pattern_2 = r'\n'

  for text in lst_files:
    text = re.sub(pattern, '', ''.join(text))
    text = re.sub(pattern_2, '', text)
    list_del_speakers.append(text)

  return list_del_speakers

In [ ]:
# Функция для создания списков с данными из файлов .txt
def create_data_list(path, dir):

  list_annotated = os.listdir(f'{path}{dir}/annotated/')      # Список с названиями файлов, нужно для колонки name
  annotated_speaker =  read_files(f'{path}{dir}/annotated/')  # Список текстов из папки annotated
  annotated_not_speaker = del_speakers(annotated_speaker)     # Список без спикеров из папки annotated

  task_speaker = read_files(f'{path}{dir}/task/')             # Список текстов из папки task
  task_not_speaker = del_speakers(task_speaker)               # Список без спикеров из папки task

  return list_annotated, annotated_speaker, annotated_not_speaker, task_speaker, task_not_speaker

In [ ]:
# Функция которая создает три pandas таблицы: real, syntatic, translate

def create_table(path):

  list_path = os.listdir(path)

  df_real= pd.DataFrame(columns=['annotated_speaker',
                           'annotated_not_speaker',
                           'task_spearker',
                           'task_not_speaker',
                           'type'
                           'name'])

  df_syntatic= pd.DataFrame(columns=['annotated_speaker',
                            'annotated_not_speaker',
                            'task_spearker',
                            'task_not_speaker',
                            'type'
                            'name'])

  df_translate= pd.DataFrame(columns=['annotated_speaker',
                            'annotated_not_speaker',
                            'task_spearker',
                            'task_not_speaker',
                            'type'
                            'name'])

  for dir in list_path:
    # Если это папка real
    if dir == 'real':

      list_annotated, annotated_speaker, annotated_not_speaker, task_speaker, task_not_speaker = create_data_list(path, dir)

      # Создайте пустой список для хранения строк DataFrame
      data_rows = []

      for i in range(len(annotated_speaker)):
        row = {'annotated_speaker': annotated_speaker[i],
                'annotated_not_speaker': annotated_not_speaker[i],
                'task_speaker': task_speaker[i],
                'task_not_speaker': task_not_speaker[i],
                'type': 'real',
                'name': list_annotated[i][:-14]}
        data_rows.append(row)

      # Создайте DataFrame, используя pandas.concat
      df_real = pd.concat([pd.DataFrame(data_rows)], ignore_index=True)



    # Если это папка syntatic
    elif dir == 'syntatic':

      list_annotated, annotated_speaker, annotated_not_speaker, task_speaker, task_not_speaker = create_data_list(path, dir)

      # Создайте пустой список для хранения строк DataFrame
      data_rows = []

      for i in range(len(annotated_speaker)):
        row = {'annotated_speaker': annotated_speaker[i],
                'annotated_not_speaker': annotated_not_speaker[i],
                'task_speaker': task_speaker[i],
                'task_not_speaker': task_not_speaker[i],
                'type': 'syntatic',
                'name': list_annotated[i][:-14]}
        data_rows.append(row)

      # Создайте DataFrame, используя pandas.concat
      df_syntatic = pd.concat([pd.DataFrame(data_rows)], ignore_index=True)


    # Если это папка с подпапками
    if dir == 'translate':

      list_subdirs = os.listdir(os.path.join(path, dir))    # ['translated_PL', 'translated_JP']

      for subdir in list_subdirs:

        join_subdir = f'{dir}/{subdir}/'
        list_annotated, annotated_speaker, annotated_not_speaker, task_speaker, task_not_speaker = create_data_list(path, join_subdir)
        data_rows = []

        for i in range(len(annotated_speaker)):
          row = {'annotated_speaker': annotated_speaker[i],
                  'annotated_not_speaker': annotated_not_speaker[i],
                  'task_speaker': task_speaker[i],
                  'task_not_speaker': task_not_speaker[i],
                  'type': 'translate',
                  'name': list_annotated[i][:-17]}
          data_rows.append(row)

        # Проверьте, пустая ли таблица df
        if df_translate.empty:
            # Если она пустая
            df_translate = pd.DataFrame(data_rows)
        else:
            # Если таблица уже содержит данные
            df_translate = pd.concat([df_translate, pd.DataFrame(data_rows)], ignore_index=True)

  return df_real, df_syntatic, df_translate

In [ ]:
# Путь к папке где лежат папки: real, syntatic, translate
path = './Datasets/'

In [ ]:
# Создаем три pandas таблицы с данными
df_real, df_syntatic, df_translate = create_table(path)

In [ ]:
# Объединение DataFrame по столбцам
result_df = pd.concat([df_real, df_syntatic, df_translate], axis=0)

In [ ]:
# Сохранение DataFrame в формате .csv
df_real.to_csv('df_real.csv', index=False, encoding='utf-8-sig')
df_syntatic.to_csv('df_syntatic.csv', index=False, encoding='utf-8-sig')
df_translate.to_csv('df_translate.csv', index=False, encoding='utf-8-sig')
result_df.to_csv('df_total.csv', index=False, encoding='utf-8-sig')

# Сохранение DataFrame в формате .xlsx
df_real.to_excel('df_real.xlsx', index=False)
df_syntatic.to_excel('df_syntatic.xlsx', index=False)
df_translate.to_excel('df_translate.xlsx', index=False)
result_df.to_excel('df_total.xlsx', index=False)